# LIMPEZA DE DADOS EM UM DATABASE.

**1. Definição do Problema**

Um banco tem uma base da dados, realizando uma Análise podemos observar que há uma alta cardinalidade.Desta forma precisamos tratar estes dados!

**2. Importando os dados do banco, podemos observar se o pagamento foi realizado ou não.**
Para importar utilizamos a biblioteca pandas que possibilita a importação do arquivo para manipularmos os dados.

In [ ]:
import pandas as pd

In [ ]:
base=pd.read_excel(r"C:\Users\Administrador\Downloads\ChavesClientes.xlsx",sheet_name = "base")

In [14]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


**3. Verificando a cardionalidade dos dados.**

In [15]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

Podemos observar que os dados estão misturados e bagunçados. 
Ao entrar em contato com o responsável pela área, foi informado o que cada coluna representa. Assim sendo:
Ao mapear com o responsável pela área, ele informou o que representa cada coluna:
   - ChaveSituacao: formado por:
    - Idade do cliente (a idade mínima para ser cliente é 18 anos)
    - Gênero do cliente:
        - M: Masculino
        - F: Feminino
    - Estado civil do cliente:
        - S: solteiro
        - C: casado
        - D: divorciado
        - V: viúvo
    - ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
    - Categoria VIP: categoria do cliente VIP (caso exista)
        - Alpha
        - Beta

In [ ]:
base['Idade'] = base.ChaveSituacao.str[:2] ## utilizando as duas primeiras strings para formar a idade.
base['Genero'] = base.ChaveSituacao.str[2:3] ## utilizando a terceira strinig para formar o Gênero.
base['EstadoCivil'] = base.ChaveSituacao.str[-1] ## utilizando a ultima string para formar o Estado Civil.

In [30]:
base.groupby(["Pagamento","EstadoCivil"])["Pagamento"].count()

Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

Verificando os dados podemos notar que agora estão com Categorias adequadas e por exemplo, que todos os viúvos pagaram suas dívidas.

In [31]:
base['Categoria'] = base.CatCliente.str.split('-').str.get(0) ## Enviando dados antes do - para Categoria.
base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)  ## Enviando dados depois do - para CatVip.

In [32]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Risco,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,A,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,B-,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,B,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,C-,Black,NaN


Como agora para realizar a separação dos dados não conseguimos utilizar apenas o critério de um traço, iremos utilizar o Regex que possibilita identificar as cadeias de forma mais específica.

In [33]:
import re

In [34]:
base['Risco'] = base.ClassRisco.apply(lambda x: re.findall('^[A-Z][^A-Za-z]?',x)[0]) ##filtrando apenas a primeira letra de ClassRisco.

In [35]:
base.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Risco          20 non-null     object
 9   Categoria      20 non-null     object
 10  CatVIP         8 non-null      object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


In [36]:
base['Idade'] = pd.to_numeric(base['Idade']) ## transformando a Coluna Idade para o tipo Númerico. 

In [37]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Risco          20 non-null     object
 9   Categoria      20 non-null     object
 10  CatVIP         8 non-null      object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


In [38]:
base.loc[base.CatVIP.isnull(),"CatVIP"]="Comum" #Atribuindo valor para Cliente Comum em CatVIP.

In [39]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Risco          20 non-null     object
 9   Categoria      20 non-null     object
 10  CatVIP         20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


In [40]:
base.groupby(["Pagamento","Risco"])["Pagamento"].count() # Podemos observar que clientes do tipo C não realizaram pagamento.

Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64

In [41]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Risco,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,A,Black,Comum
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,B-,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,B,Black,Comum
4,5,26FD,C-Amarelo,Black,0,26,F,D,C-,Black,Comum


In [42]:
base.drop(["ChaveSituacao"], axis=1, inplace=True) # Excluindo todas as Colunas que estavam com dados misturados.

In [45]:
base.drop(["CatCliente"], axis=1, inplace=True)

In [ ]:
base.drop(["ChaveSituacao"], axis=1, inplace=True)

In [46]:
base.head()

,ID,Pagamento,Idade,Genero,EstadoCivil,Risco,Categoria,CatVIP
0,1,1,32,F,C,C,Basic,Alpha
1,2,1,25,M,V,A,Black,Comum
2,3,1,27,M,V,B-,Basic,Beta
3,4,0,26,F,D,B,Black,Comum
4,5,0,26,F,D,C-,Black,Comum
